In [ ]:
# Instalando o modulo do gemini AI
# -U -> Update
!pip install -U -q google-generativeai

In [2]:
import json
import pandas as pd
import textwrap
from IPython.display import display
from IPython.display import Markdown

In [3]:
from google.colab import userdata
import google.generativeai as genai

API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=API_KEY)

In [4]:
#Listando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.05, # Pelo modelo se comportar como uma API, optou-se por usar
}

In [52]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

In [53]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings)

In [54]:
examples = pd.DataFrame([
    {
        "target_midia" : "Manga",
        "reference" : "Oppenheimer (Filme)",
        "num_resp" : 2,
        "resultado" : [{
            "titulo": "Pluton",
            "autor": "Naoki Urasawa",
            "genero": "Drama, Ficção Científica, História",
            "sinopse": "Um jovem cientista alemão se junta ao projeto Manhattan para desenvolver a bomba atômica durante a Segunda Guerra Mundial.",
          },
          {
            "titulo": "This is How I Kill Your King",
            "autor": "Warren Ellis e Giannis Milonogiannis",
            "genero": "Drama, Ficção Científica, Thriller",
            "sinopse": "Um agente do governo americano investiga uma série de assassinatos que podem estar relacionados ao desenvolvimento da bomba atômica.",
          }]
    }, {
        "target_midia" : "Serie",
        "reference" : "O show de truman (Filme)",
        "num_resp" : 2,
        "resultado" : [{
            "titulo": "Black Mirror",
            "ano": "2011-2019",
            "genero": "Ficção Científica, Drama",
            "sinopse": "Uma série de antologia que explora o lado obscuro da tecnologia e seus impactos na sociedade.",
          },
          {
            "titulo": "Westworld",
            "ano": "2016-presente",
            "genero": "Ficção Científica, Drama",
            "sinopse": "Um parque temático futurista onde androides vivem em um mundo do Velho Oeste, mas as coisas começam a dar errado.",
          }]
    }, {
        "target_midia" : "Filme",
        "reference" : "Blame! (manga)",
        "num_resp" : 4,
        "resultado" : [{
            "titulo": "Ghost in the Shell",
            "ano": "1995",
            "genero": "Ação, Ficção Científica",
            "sinopse": "Um cyborg policial investiga um hacker que está implantando memórias falsas em pessoas."
          },
          {
            "titulo": "O Gigante de Ferro",
            "ano": "1999",
            "genero": "Animação, Aventura",
            "sinopse": "Um menino faz amizade com um robô gigante que é caçado pelo governo."
          },
          {
            "titulo": "Akira",
            "ano": "1988",
            "genero": "Animação, Ação",
            "sinopse": "Em uma Tóquio distópica, um jovem motociclista se envolve em uma conspiração governamental."
          },
          {
            "titulo": "Blade Runner 2049",
            "ano": "2017",
            "genero": "Ficção Científica, Thriller",
            "sinopse": "Um policial replicante investiga o mistério de uma criança nascida de uma mãe humana."
          }]
    }, {
        "target_midia" : "Filme",
        "reference" : "icjqicjpjcpj",
        "num_resp" : 2,
        "resultado" : {"error": "O elemento de referência não foi reconhecido, por favor tente novamente."},
    }, {
        "target_midia" : "",
        "reference" : "O lobo de Wallstreet",
        "num_resp" : 2,
        "resultado" : {"error": "Necessário passar um tipo de mídia valida como target, por favor tente novamente."},
    }
])

examples

,target_midia,reference,num_resp,resultado
0,Manga,Oppenheimer (Filme),2,"[{'titulo': 'Pluton', 'autor': 'Naoki Urasawa'..."
1,Serie,O show de truman (Filme),2,"[{'titulo': 'Black Mirror', 'ano': '2011-2019'..."
2,Filme,Blame! (manga),4,"[{'titulo': 'Ghost in the Shell', 'ano': '1995..."
3,Filme,icjqicjpjcpj,2,{'error': 'O elemento de referência não foi re...
4,,O lobo de Wallstreet,2,{'error': 'Necessário passar um tipo de mídia ...


In [55]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> '))

In [56]:
def print_infos( model_resp ):

  #print(model_resp)

  filter_resp = model_resp.replace("```", "").replace("json", "")
  filter_resp = json.loads(filter_resp)

  #print(filter_resp)

  filter_resp = filter_resp["resultado"]

  #print(filter_resp)

  if isinstance(filter_resp, dict):
    if 'error' in filter_resp:
      display( to_markdown("## Um erro foi identificado :( "))
      display( to_markdown(f"> {filter_resp['error']}"))

  else:
    display(to_markdown("## Talvez você goste de : \n"))
    for content in filter_resp:
      for key, value in content.items():
        #print(f"**{key}** : {value}")
        display(to_markdown(f"**{key}** : {value}"))
      print("---" * 20)

In [57]:
def process_data_and_print_response(media_type, reference):
  #mas a ideia é recomendar obras diferentes, então priorize obras que não sejam ambientadas no mesmo universo de {reference}.

  # send_query_to_search = f"""
  #   Se comporte como se fosse um sistema backend que devolve um json com dados de uma API.
  #   Devolva uma lista com 5 {media_type} relacionado a tema de {reference}, pode ser do mesmo universo
  #   A resposta deve ser uma lista JSON com os seguintes parametros: ['Nome', 'Ano', 'Sinopse', 'Gemero'].
  #   Caso o conteúdo de base não exista ou não seja encontrado devolva um json com uma mensagem explicando o problema e pedindo para o usuário tentar novamente, devolvendo em uma chave error
  # """

  send_query_to_search = f"""
    Tendo como base os exemplos : {examples}, responda como se fosse uma API em JSON para o seguinte caso :

    target_midia : {media_type},
    reference : {reference},
    num_resp : 3,

    resultado :
  """

  response = model.generate_content(send_query_to_search)
  model_resp = response.text

  print_infos( model_resp )



In [22]:
MEDIA_TYPES = ["Filme","Série", "Livro", "Manga", "Video game"]

In [58]:
print("""
  Selecione o tipo de mídia :
  1. Filme;
  2. Série;
  3. Livro;
  4. Manga;
  5. Video game;
  """)

index_type = input("Selecione valor válido: ")

if index_type in ("1", "2", "3", "4", "5"):

  media_reference = input("Nome de uma obra que você gostaira de ver mais : ")

  process_data_and_print_response( MEDIA_TYPES[int(index_type) - 1], media_reference)


  Selecione o tipo de mídia :
  1. Filme;
  2. Série;
  3. Livro;
  4. Manga;
  5. Video game;
  
Selecione valor válido: 2
Nome de uma obra que você gostaira de ver mais : Invencivel 


> ## Talvez você goste de : 


> **titulo** : The Boys

> **ano** : 2019-presente

> **genero** : Super-heróis, Ação, Drama

...

> **titulo** : Peacemaker

> **ano** : 2022-presente

> **genero** : Super-heróis, Ação, Comédia

> **plataforma** : HBO Max

------------------------------------------------------------
